In [2]:
#!ipcluster start -n 8 --engines=MPI --profile='mpi' # for parallel run: start the engines using terminal
from ipyparallel import Client
rc = Client(profile='default')

In [4]:
%%px
# Import the libraries

import os
import sys
import math
import numpy as np
import pyfftw as ft 
from mpi4py import MPI
import matplotlib
import matplotlib.pyplot as plt
import pyJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse
from pyJHTDB import libJHTDB

from fft3d import FFT3Dfield_new
from EnergySpectrum import EnergySpectrum

CompositeError: one or more exceptions from call to method: execute
[0:execute]: ImportError: No module named 'fft3d'
[1:execute]: ImportError: No module named 'fft3d'
[2:execute]: ImportError: No module named 'fft3d'
[3:execute]: ImportError: No module named 'fft3d'
.... 4 more exceptions ...

In [3]:
%%px

comm = MPI.COMM_WORLD; rank = comm.Get_rank(); nproc = comm.Get_size()
if(rank==0):
    print("n_proc = "+str(nproc))
    print("rank = "+str(rank))

# pyJHTDB parameters
    
Nx = isotropic1024coarse['nx']; Ny = isotropic1024coarse['ny']; Nz = isotropic1024coarse['nz']
Lx = isotropic1024coarse['lx']; Ly = isotropic1024coarse['ly']; Lz = isotropic1024coarse['lz']
auth_key = "com.gmail.jhelsas-b854269a"
 
# Computational Domain

nx=Nx//nproc; ny=Ny; nz=Nz
nek=int(math.sqrt(2.0)/3*Nx)
time = 0.0

chkSz = 32
slabs = nx//chkSz

[stdout:1] 
n_proc = 8
rank = 0


In [4]:
%%px

vx = ft.zeros_aligned((nx,ny,nz), dtype='float32')
vy = ft.zeros_aligned((nx,ny,nz), dtype='float32')
vz = ft.zeros_aligned((nx,ny,nz), dtype='float32')

# Populate velocity field from the Database

if(rank==0):
    print("Starting the loading process")

##########################################
load_from_file = True

folder = "/home/idies/workspace/scratch"
filename = "check-isotropic1024coarse-"+str(rank)+"-(t="+str(time)+")"+".npz"
file = folder + "/" + filename

if(load_from_file):
    comm.Barrier(); t1=MPI.Wtime()
    content = np.load(file)
    if(int(content['nproc'])!=nproc):
        print("Unmatched number of processes. Must first pre-process to adequate number of process")
    vx = content['vx']
    vy = content['vy']
    vz = content['vz']
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        print("Finished loading")
        sys.stdout.write('Load from disk: {0:.2f} seconds\n'.format(t2-t1))
else:
    comm.Barrier(); t1=MPI.Wtime()
    lJHTDB = libJHTDB()
    lJHTDB.initialize()
    ud = []
    for k in range(slabs):
        if(rank==0):
            print(k)
        start = np.array([rank*nx+k*chkSz, 0, 0],dtype=np.int)
        width = np.array([chkSz,ny,nz],dtype=np.int)
        #start = np.array([ 0, 0, rank*nx+k*chkSz],dtype=np.int)
        ud.append(lJHTDB.getRawData(time,start,width, 
                                    data_set = 'isotropic1024coarse',
                                    getFunction = 'Velocity') )
    
    lJHTDB.finalize()
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        print("Finished loading")
        sys.stdout.write('Load field from database: {0:.2f} seconds\n'.format(t2-t1))
    
    u = np.concatenate(ud,axis=2)
    comm.Barrier(); t1=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Concatenate: {0:.2f} seconds\n'.format(t1-t2))
    
    rsh = np.transpose(u,(2,1,0,3))
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Transpose: {0:.2f} seconds\n'.format(t2-t1))
    ##########################################
    
    vx[:,:,:] = rsh[:,:,:,0]
    vy[:,:,:] = rsh[:,:,:,1]
    vz[:,:,:] = rsh[:,:,:,2]
    comm.Barrier(); t1=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Splitting: {0:.2f} seconds\n'.format(t1-t2))
    
    np.savez(file,vx=vx,vy=vy,vz=vz,nproc=nproc)
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Caching the data: {0:.2f} seconds\n'.format(t2-t1))


if(rank==0):
    print("vx shape = "+str(vx.shape))       

[stdout:1] 
Starting the loading process
Finished loading
Load from disk: 292.18 seconds
vx shape = (128, 1024, 1024)


In [6]:
%%px

comm.Barrier(); t1=MPI.Wtime()
# Get wavenumber:
myEnergySpc=EnergySpectrum()
kx,ky,kz=myEnergySpc.FindWavenumber(nx,ny,nz,rank)
k2=np.zeros((nx,ny,(nz//2)+1), dtype='float32')
np.copyto(k2,kx*kx+ky*ky+kz*kz)
k2[0,0,0]=1e-6

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate all wavenumbers: {0:.2f} seconds\n'.format(t2-t1))

[stdout:1] Calculate all wavenumbers: 29.12 seconds


In [7]:
%%px 

fft = FFT3Dfield_new(nx,ny,nz,nproc,rank)

In [8]:
%%px

cvx = ft.zeros_aligned((nx,ny,1+(nz//2)), dtype='complex64')
cvy = ft.zeros_aligned((nx,ny,1+(nz//2)), dtype='complex64')
cvz = ft.zeros_aligned((nx,ny,1+(nz//2)), dtype='complex64')

In [9]:
%%px
comm.Barrier(); t1=MPI.Wtime()

cvx[:,:,:] = fft.forward3Dfft(vx,nx,ny,nz,nproc,rank)
cvy[:,:,:] = fft.forward3Dfft(vy,nx,ny,nz,nproc,rank)
cvz[:,:,:] = fft.forward3Dfft(vz,nx,ny,nz,nproc,rank)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate 3D spatial FFT: {0:.2f} seconds\n'.format(t2-t1))

[stdout:1] Calculate 3D spatial FFT: 185.30 seconds


In [10]:
%%px
comm.Barrier(); t1=MPI.Wtime()

cwx=0.5*np.complex64(0.0+1.0j)*(ky*cvz-kz*cvy)
cwy=0.5*np.complex64(0.0+1.0j)*(kz*cvx-kx*cvz)
cwz=0.5*np.complex64(0.0+1.0j)*(kx*cvy-ky*cvx)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate vorticity in k space: {0:.2f} seconds\n'.format(t2-t1))

[stdout:1] Calculate vorticity in k space: 98.67 seconds


In [12]:
%%px
comm.Barrier(); t1=MPI.Wtime()

cSxx=np.complex64(0.0+1.0j)*(kx*cvx)
cSyy=np.complex64(0.0+1.0j)*(ky*cvy)
cSzz=np.complex64(0.0+1.0j)*(kz*cvz)

cSxy=0.5*np.complex64(0.0+1.0j)*(kx*cvy+ky*cvx)
cSyz=0.5*np.complex64(0.0+1.0j)*(ky*cvz+kz*cvy)
cSxz=0.5*np.complex64(0.0+1.0j)*(kx*cvz+kz*cvx)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate strain-rate in k space: {0:.2f} seconds\n'.format(t2-t1))

[stdout:1] Calculate strain-rate in k space: 210.52 seconds


In [ ]:
%%px
comm.Barrier(); t1=MPI.Wtime()

wx = ft.zeros_aligned((nx,ny,nz), dtype='float32')
wy = ft.zeros_aligned((nx,ny,nz), dtype='float32')
wz = ft.zeros_aligned((nx,ny,nz), dtype='float32')

Sxx = ft.zeros_aligned((nx,ny,nz), dtype='float32')
Syy = ft.zeros_aligned((nx,ny,nz), dtype='float32')
Szz = ft.zeros_aligned((nx,ny,nz), dtype='float32')

Sxy = ft.zeros_aligned((nx,ny,nz), dtype='float32')
Syz = ft.zeros_aligned((nx,ny,nz), dtype='float32')
Sxz = ft.zeros_aligned((nx,ny,nz), dtype='float32')

cwx[kz==(Nz//2)] = 0.
cwy[kz==(Nz//2)] = 0.
cwz[kz==(Nz//2)] = 0.

cSxx[kz==(Nz//2)] = 0.
cSyy[kz==(Nz//2)] = 0.
cSzz[kz==(Nz//2)] = 0.

cSxy[kz==(Nz//2)] = 0.
cSyz[kz==(Nz//2)] = 0.
cSxz[kz==(Nz//2)] = 0.

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Alocating memory in real space and cleaning odd-ball wavenumber: {0:.2f} seconds\n'.format(t2-t1))

In [16]:
%%px
comm.Barrier(); t1=MPI.Wtime()

wx[:,:,:]=fft.backward3Dfft(cwx,nx,ny,nz,nproc,rank)
wy[:,:,:]=fft.backward3Dfft(cwy,nx,ny,nz,nproc,rank)
wz[:,:,:]=fft.backward3Dfft(cwz,nx,ny,nz,nproc,rank)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate 3D spatial inverse FFT: {0:.2f} seconds\n'.format(t2-t1))

[stdout:1] Calculate 3D spatial inverse FFT: 164.41 seconds


In [17]:
%%px
comm.Barrier(); t1=MPI.Wtime()

Sxx[:,:,:]=fft.backward3Dfft(cSxx,nx,ny,nz,nproc,rank)
Syy[:,:,:]=fft.backward3Dfft(cSyy,nx,ny,nz,nproc,rank)
Szz[:,:,:]=fft.backward3Dfft(cSzz,nx,ny,nz,nproc,rank)

Sxy[:,:,:]=fft.backward3Dfft(cSxy,nx,ny,nz,nproc,rank)
Syz[:,:,:]=fft.backward3Dfft(cSyz,nx,ny,nz,nproc,rank)
Sxz[:,:,:]=fft.backward3Dfft(cSxz,nx,ny,nz,nproc,rank)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate 3D spatial inverse FFT: {0:.2f} seconds\n'.format(t2-t1))

[stdout:1] Calculate 3D spatial inverse FFT: 423.95 seconds


In [ ]:
%%px
comm.Barrier(); t1=MPI.Wtime()

w2 = 2.0*(wx**2+wy**2+wz**2)
S2 = Sxx**2+Syy**2+Sz**2+2.0*(Sxy**2+Syz**2+Sxz**2)

slabEnstrophy  = np.average(w2)
slabStrainrate = np.average(S2)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate 3D spatial inverse FFT: {0:.2f} seconds\n'.format(t2-t1))

In [ ]:
%%px

avgStrain=np.zeros(1,dtype='float32')

comm.Reduce([slabStrainrate,MPI.REAL],[avgStrain,MPI.REAL],op=MPI.SUM)
avgStrain = avgStrain[0]/nproc
avgStrain = comm.bcast(avgStrain, root=0)

In [ ]:
%%px

avgEnstrophy=np.zeros(1,dtype='float32')

comm.Reduce([slabEnstrophy,MPI.REAL],[avgEnstrophy,MPI.REAL],op=MPI.SUM)
avgEnstrophy = avgEnstrophy[0]/nproc
avgEnstrophy = comm.bcast(avgEnstrophy, root=0)

In [ ]:
%%px

cacheEnstrophyData = False
loadEnstrophyFromCache = True

folder = "/home/idies/workspace/scratch"
filename = "enstrophy-newFFT-isotropic1024coarse-"+str(rank)+"-(t="+str(time)+")"+".npz"
file = folder + "/" + filename

if(loadEnstrophyFromCache):
    comm.Barrier(); t1=MPI.Wtime()
    content = np.load(file)
    
    w2 = ft.zeros_aligned((nx,ny,nz), dtype='float32')
    
    if(int(content['nproc'])!=nproc):
        print("Unmatched number of processes. Must first pre-process to adequate number of process")
    w2[:,:,:] = content['w2']
    
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        print("Finished loading")
        sys.stdout.write('Load from disk: {0:.2f} seconds\n'.format(t2-t1))

if(cacheEnstrophyData):
    
    comm.Barrier(); t1=MPI.Wtime()
    np.savez(file,w2=w2,nproc=nproc)
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Caching the data: {0:.2f} seconds\n'.format(t2-t1))

In [ ]:
%%px

cacheEnstrophyData = False
loadEnstrophyFromCache = True

folder = "/home/idies/workspace/scratch"
filename = "strainrate-newFFT-isotropic1024coarse-"+str(rank)+"-(t="+str(time)+")"+".npz"
file = folder + "/" + filename

if(loadEnstrophyFromCache):
    comm.Barrier(); t1=MPI.Wtime()
    content = np.load(file)
    
    w2 = ft.zeros_aligned((nx,ny,nz), dtype='float32')
    
    if(int(content['nproc'])!=nproc):
        print("Unmatched number of processes. Must first pre-process to adequate number of process")
    w2[:,:,:] = content['w2']
    
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        print("Finished loading")
        sys.stdout.write('Load from disk: {0:.2f} seconds\n'.format(t2-t1))

if(cacheEnstrophyData):
    
    comm.Barrier(); t1=MPI.Wtime()
    np.savez(file,w2=w2,nproc=nproc)
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Caching the data: {0:.2f} seconds\n'.format(t2-t1))